In [6]:
import torch
from torch.autograd import Variable
import torch.autograd
import torch.nn as nn
import torch.nn.functional as F
import data_feeder as df
import numpy



class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        pretrained = False;
        if (pretrained == True):
            self.model = torchvision.models.vgg16(pretrained=True)
            for param in self.model.parameters():
                param.requires_grad = False
            self.model.fc = nn.Linear(128, 1) 
        else:
            self.model = nn.Sequential(
                    nn.Conv2d(3, 8, 5, stride = 2) 
                    ,nn.BatchNorm2d(8)
                        ,nn.ReLU()
                        ,nn.Conv2d(8, 16, 5, stride = 2)
                        ,nn.BatchNorm2d(16)
                            ,nn.ReLU()
                        ,nn.Conv2d(16, 32, 5)
                            ,nn.BatchNorm2d(32)
                            ,nn.ReLU()
                            ,nn.Conv2d(32, 48, 5)
                            ,nn.BatchNorm2d(48)
                            ,nn.ReLU()
                            ,nn.Conv2d(48, 1, 5)
                )
            
           # self.fc1 = nn.Linear(24, 64)
           # self.fc2 = nn.Linear(64, 1)
              #                  ,nn.Linear(48 * 48 * 24, 128)
             #                       ,nn.BatchNorm1d(128)
            #                        ,nn.ReLU()
           #                         ,nn.Linear(128, 1)
                
    
    def forward(self, input_data):
        return self.model(input_data)
    
 #   def _bias_init(self, b):
  #      b.bias = torch.nn.Parameter(torch.zeros(len(b.bias)))

  #  def _weight_init(self, m):
   #     size = m.weight.size()
    #    fan_out = size[0]  # number of rows
    #    fan_in = size[1]  # number of columns
    #    variance = np.sqrt(2.0 / (fan_in + fan_out))
    #    m.weight.data.normal_(0.0, variance)
    @property
    def is_cuda(self):
        return next(self.parameters()).is_cuda
    
class Solver():
    def __init__(self):
        super(Solver, self).__init__()
        
    def loss(self, pred, labels):
        return (pred - labels).pow(2).sum() / len(labels)
    
    def accuracy(self, pred, labels):
        acc = 0
        for i in range(len(labels)):
            predict = pred[i].data.cpu().numpy()[0]
            begin = labels[i].data.cpu().numpy() * 1.1
            end = labels[i].data.cpu().numpy() * 0.9
          #  print(predict)
            if predict >= 0:
                if predict >= end and predict <= begin:
                    acc+=1
            else:
                if predict <= end and predict >= begin:
                    acc+=1
                
        return acc/len(labels)
    
    def train(self, train_data, num_epochs = 15, learning_rate = 1e-1, batch_size = 5, pretrained = False):

            train_loader = torch.utils.data.DataLoader(dataset = train_data,

                                               batch_size=batch_size, 

                                               shuffle=True, num_workers = 0)
       #    print(len(train_loader))
            cnn = Model()
            optimizer = torch.optim.Adam(cnn.parameters(), lr = learning_rate)
            loss_history = []
            if torch.cuda.is_available():
                cnn.cuda()
            for i in range(num_epochs):
                for j, (images, labels) in enumerate(train_loader):
                    images = Variable(images.float())
                    labels = Variable(labels.float())
                    labels *= 5
                    if cnn.is_cuda:
                        inputs, targets = inputs.cuda(), targets.cuda()
                    optimizer.zero_grad()
                    pred = cnn(images)
                    pred *= 5
                   # print(pred)
                    loss = self.loss(pred, labels)
                    loss_history.append(loss)
                    cnn.zero_grad()
                    loss.backward()
                    optimizer.step()
                       
                    if (j+1) % 10 == 0:

                        print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f, Acc: %4f' 

                           %(i+1, num_epochs, j+1, len(train_loader)//batch_size, loss.data[0], self.accuracy(pred, labels)))
                epoch_loss = 0
                for j in range(1, len(train_loader)):
                    epoch_loss += loss_history[-j]
                print ('EPOCH LOSS: %.4f' % (epoch_loss/len(train_loader)))
                       # print(pred, labels)
                        
train_data, val_data = df.getDrivingData('race1516207298.txt')

solver = Solver()
solver.train(train_data)         
    
        
    
    
    
    

Epoch [1/15], Iter [10/15] Loss: 175.7288, Acc: 0.000000
Epoch [1/15], Iter [20/15] Loss: 23.7623, Acc: 0.000000
Epoch [1/15], Iter [30/15] Loss: 36.6576, Acc: 0.000000
Epoch [1/15], Iter [40/15] Loss: 13.6182, Acc: 0.000000
Epoch [1/15], Iter [50/15] Loss: 2.3508, Acc: 0.000000
Epoch [1/15], Iter [60/15] Loss: 2.1028, Acc: 0.200000
Epoch [1/15], Iter [70/15] Loss: 1.2715, Acc: 0.000000
EPOCH LOSS: 462.9911
Epoch [2/15], Iter [10/15] Loss: 1.4760, Acc: 0.000000
Epoch [2/15], Iter [20/15] Loss: 0.4256, Acc: 0.000000
Epoch [2/15], Iter [30/15] Loss: 0.6708, Acc: 0.000000
Epoch [2/15], Iter [40/15] Loss: 0.8687, Acc: 0.000000
Epoch [2/15], Iter [50/15] Loss: 0.2757, Acc: 0.000000
Epoch [2/15], Iter [60/15] Loss: 1.5095, Acc: 0.000000
Epoch [2/15], Iter [70/15] Loss: 0.6418, Acc: 0.000000
EPOCH LOSS: 0.6705
Epoch [3/15], Iter [10/15] Loss: 0.3155, Acc: 0.000000
Epoch [3/15], Iter [20/15] Loss: 1.5016, Acc: 0.000000
Epoch [3/15], Iter [30/15] Loss: 0.7631, Acc: 0.000000
Epoch [3/15], Iter [